In [11]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [12]:
#Adapter layer
def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter = sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket,prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files
    

In [13]:
#Application layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'),closing_price_eur=('closing_price','min'),minimum_price_eur=('MinPrice','min'),maximum_price_eur=('MaxPrice','max'),daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'],inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [14]:
#Application layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
    return return_date_list

In [15]:
# main function entrypoint

def main():
    #parameters/confirgurations
    #later read config
    arg_date = '2021-10-21'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-868686'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report'
    trg_format = '.parquet'
    
    #Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)

    #run application
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

    

In [16]:
#run

main()

#Get opening price per ISIN and day

## Get closing pirce per ISIN and day

##Aggregations

## Percent Change Prev Closing

## Read the uploaded file

In [17]:
trg_bucket = 'xetra-868686'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report20211013_143021.parquet
xetra_daily_report20211013_203606.parquet
xetra_daily_report20211020_120112.parquet
xetra_daily_report20211022_102436.parquet


In [20]:
prq_obj = bucket_trg.Object(key='xetra_daily_report20211022_102436.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-10-21,9.00,9.02,8.90,9.02,1060,-0.11
1,AT00000FACC2,2021-10-22,9.00,9.04,8.91,9.04,933,0.22
2,AT0000606306,2021-10-21,24.70,24.50,24.50,24.76,1302,-0.73
3,AT0000606306,2021-10-22,24.84,25.00,24.84,25.00,1110,2.04
4,AT0000609607,2021-10-21,14.22,13.84,13.66,14.22,415,-4.95
...,...,...,...,...,...,...,...,...
6164,XS2284324667,2021-10-21,35.07,34.61,34.61,35.11,3114,-1.42
6165,XS2284324667,2021-10-22,34.90,34.92,34.90,35.24,1661,0.90
6166,XS2314659447,2021-10-22,8.77,8.58,8.58,8.77,951,-5.10
6167,XS2314660700,2021-10-21,17.82,17.21,17.21,17.82,595,-2.48
